In [1]:
import sys
sys.path = ['', '/home/erik_jones313/miniconda2/envs/erik/lib/python27.zip', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/plat-linux2', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/lib-tk', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/lib-old', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/lib-dynload', '/home/erik_jones313/.local/lib/python2.7/site-packages', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/site-packages']


In [2]:
import numpy as np
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


In [3]:
import data_utils
import glove_utils
import models
import display_utils
from goog_lm import LM

In [4]:
import lm_data_utils
import lm_utils

In [5]:
from im_utils import load_model, save_model

In [6]:
np.random.seed(1001)
tf.set_random_seed(1001)

In [7]:
#MAY NOT BE WORKING
%load_ext autoreload
%autoreload 2

The autoreload module is not an IPython extension.


UsageError: Line magic function `%autoreload` not found.


In [8]:
VOCAB_SIZE  = 10000
with open('aux_files/dataset_%d.pkl' %VOCAB_SIZE, 'rb') as f:
    dataset = pickle.load(f)

In [9]:
doc_len = [len(dataset.test_seqs2[i]) for i in 
           range(len(dataset.test_seqs2))]

In [10]:
dist_mat = np.load('aux_files/dist_counter_%d.npy' %VOCAB_SIZE)
# Prevent returning 0 as most similar word because it is not part of the dictionary
dist_mat[0,:] = 100000
dist_mat[:,0] = 100000

skip_list = np.load('aux_files/missed_embeddings_counter_%d.npy' %VOCAB_SIZE)

### Demonstrating how we find the most similar words

In [11]:
for i in range(300, 305):
    src_word = i
    nearest, nearest_dist = glove_utils.pick_most_similar_words(src_word, dist_mat,20, 0.5)
        
    print('Closest to `%s` are:' %(dataset.inv_dict[src_word]))
    for w_id, w_dist in zip(nearest, nearest_dist):
          print(' -- ', dataset.inv_dict[w_id], ' ', w_dist)

    print('----')

Closest to `later` are:
(' -- ', 'subsequent', ' ', 0.18323109771400015)
(' -- ', 'subsequently', ' ', 0.1867195991340007)
(' -- ', 'afterward', ' ', 0.2509214012219996)
(' -- ', 'afterwards', ' ', 0.2576958961479996)
(' -- ', 'thereafter', ' ', 0.2741981096589998)
(' -- ', 'after', ' ', 0.34520261237799876)
(' -- ', 'then', ' ', 0.36472839338299834)
(' -- ', 'following', ' ', 0.4833073676040003)
----
Closest to `takes` are:
(' -- ', 'pick', ' ', 0.31130546563200046)
(' -- ', 'taking', ' ', 0.42471158462800007)
(' -- ', 'picked', ' ', 0.48527412495900113)
----
Closest to `instead` are:
(' -- ', 'however', ' ', 0.3475382865829997)
(' -- ', 'alternately', ' ', 0.4439627395600003)
(' -- ', 'nevertheless', ' ', 0.477163975792001)
----
Closest to `seem` are:
(' -- ', 'seems', ' ', 0.007052995653001215)
(' -- ', 'appears', ' ', 0.32837244735200044)
(' -- ', 'looks', ' ', 0.33534638306400066)
----
Closest to `beautiful` are:
(' -- ', 'gorgeous', ' ', 0.019236443661999614)
(' -- ', 'wonderful'

### Preparing the dataset

In [12]:
max_len = 100 #CHANGED FROM 250
train_x = pad_sequences(dataset.train_seqs2, maxlen=max_len, padding='post')
train_y = np.array(dataset.train_y)
test_x = pad_sequences(dataset.test_seqs2, maxlen=max_len, padding='post')
test_y = np.array(dataset.test_y)

### Loading the sentiment analysis model

In [13]:
"""
tf.reset_default_graph()
if tf.get_default_session():
    sess.close()
sess = tf.Session()
"""
batch_size = 1
lstm_size = 128
#max_len =  100
keras = True
"""
if not keras:
    with tf.variable_scope('imdb', reuse=False):
        model = models.SentimentModel(batch_size=batch_size,
                               lstm_size = lstm_size,
                               max_len = max_len,
                               embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
    saver = tf.train.Saver()
    saver.restore(sess, './models/imdb_model')
"""

"\nif not keras:\n    with tf.variable_scope('imdb', reuse=False):\n        model = models.SentimentModel(batch_size=batch_size,\n                               lstm_size = lstm_size,\n                               max_len = max_len,\n                               embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)\n    saver = tf.train.Saver()\n    saver.restore(sess, './models/imdb_model')\n"

## Try Attack

In [14]:
filename = 'trained_model'
model = load_model(filename = filename)
batch_model = load_model(filename = filename)
neighbour_model = load_model(filename = filename)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk
Loaded model from disk
Loaded model from disk


## Main Attack 

In [15]:
def label_match(pred, label):
    return label == 1 and pred > 0.5 or label == 0 and pred <= 0.5


In [16]:
import attacks
attacks = reload(attacks)

In [17]:
pop_size = 60
n1 = 8
goog_lm = None
"""
resuse = True
if keras:
    reuse = False
with tf.variable_scope('imdb', reuse=reuse):
    batch_model = models.SentimentModel(batch_size=pop_size,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
    
with tf.variable_scope('imdb', reuse=True):
    neighbour_model = models.SentimentModel(batch_size=n1,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
"""
ga_atttack = attacks.GeneticAtack(model, batch_model, neighbour_model, dataset, dist_mat, 
                                  skip_list,
                                  goog_lm, max_iters=30,
                                   pop_size=pop_size,
                                  
                                  n1 = n1,
                                  n2 = 4,
                                 use_lm = False, use_suffix=False, keras = True)


In [ ]:
SAMPLE_SIZE = 500
TEST_SIZE = 13
keras = True
test_idx = np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
test_len = []
for i in range(SAMPLE_SIZE):
    test_len.append(len(dataset.test_seqs2[test_idx[i]]))
print('Shortest sentence in our test set is %d words' %np.min(test_len))

test_list = []
orig_list = []
orig_label_list = []
adv_list = []
dist_list = []

for i in range(SAMPLE_SIZE):
    if i % 100 == 0:
        print("Processed {} out of {}".format(i, SAMPLE_SIZE))
    #print("STARTING: ", i)
    x_orig = test_x[test_idx[i]]
    orig_label = test_y[test_idx[i]]
    orig_preds = model.predict(x_orig[np.newaxis,:])[0][0]
    #print("DATASET SHAPE: ", test_x.shape)
    #print("SHAPE MOD: ", model.predict(x_orig[np.newaxis,:]).shape)
    #print("SHAPE OG: ", model.predict(x_orig).shape)
    #print("X_ORIG SHAPE: ", x_orig.shape)
    #print("MODIFIED X_ORIG SHAPE: ", x_orig[np.newaxis, :].shape)
    print("Orig_preds: {} Orig_label: {}".format(orig_preds, orig_label))
    # print(orig_label, orig_preds, np.argmax(orig_preds))
    if not label_match(orig_preds, orig_label):
        print('skipping wrong classifed ..')
        print('--------------------------')
        continue
    x_len = np.sum(np.sign(x_orig))
    #if x_len >= 100:
    #    print('skipping too long input.. length: ', x_len)
    #    print('--------------------------')
    #    continue
    if np.abs(orig_preds - 0.5) < 0.25:
        print('skipping low confidence .. \n-----\n')
        continue
    print('****** ', len(test_list) + 1, ' ********')
    test_list.append(test_idx[i])
    orig_list.append(x_orig)
    target_label = 1 if orig_label == 0 else 0
    orig_label_list.append(orig_label)
    #print("Entering attack with x_orig, target: ({}, {}) ".format(orig_preds, target_label))
    x_adv = ga_atttack.attack( x_orig, target_label)
    adv_list.append(x_adv)
    if x_adv is None:
        print('%d failed' %(i+1))
        dist_list.append(100000)
    else:
        num_changes = np.sum(x_orig != x_adv)
        print('%d - %d changed.' %(i+1, num_changes))
        dist_list.append(num_changes)
        # display_utils.visualize_attack(sess, model, dataset, x_orig, x_adv)
    print('--------------------------')
    if (len(test_list)>= TEST_SIZE):
        break

Shortest sentence in our test set is 27 words
Processed 0 out of 500
Orig_preds: 0.582471370697 Orig_label: 1
skipping low confidence .. 
-----

Orig_preds: 0.483344495296 Orig_label: 1
skipping wrong classifed ..
--------------------------
Orig_preds: 0.61414116621 Orig_label: 0
skipping wrong classifed ..
--------------------------
Orig_preds: 0.724751234055 Orig_label: 1
skipping low confidence .. 
-----

Orig_preds: 0.716579437256 Orig_label: 0
skipping wrong classifed ..
--------------------------
Orig_preds: 0.715300321579 Orig_label: 1
skipping low confidence .. 
-----

Orig_preds: 0.584114730358 Orig_label: 1
skipping low confidence .. 
-----

Orig_preds: 0.784739196301 Orig_label: 0
skipping wrong classifed ..
--------------------------
Orig_preds: 0.541724085808 Orig_label: 0
skipping wrong classifed ..
--------------------------
Orig_preds: 0.528684556484 Orig_label: 0
skipping wrong classifed ..
--------------------------
Orig_preds: 0.516055762768 Orig_label: 0
skipping wr

## Compute Attack success rate

In [32]:
orig_len = [np.sum(np.sign(x)) for x in orig_list]
normalized_dist_list = [dist_list[i]/orig_len[i] for i in range(len(orig_list)) ]

In [33]:
SUCCESS_THRESHOLD  = 0.25
successful_attacks = [x < SUCCESS_THRESHOLD for x in normalized_dist_list]
print('Attack success rate : {:.2f}%'.format(np.mean(successful_attacks)*100))
print('Median percentange of modifications: {:.02f}% '.format(
    np.median([x for x in normalized_dist_list if x < 1])*100))
print('Mean percentange of modifications: {:.02f}% '.format(
    np.mean([x for x in normalized_dist_list if x < 1])*100))

Attack success rate : 100.00%
Median percentange of modifications: 0.00% 
Mean percentange of modifications: 0.00% 


In [40]:
import display_utils
reload(display_utils)
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(model, dataset, orig_list[visual_idx], adv_list[visual_idx])

('Original prediction: ', 0.9291805)
Original Prediction = Positive. (Confidence = 92.92) 


---------  After attack -------------
New Prediction = Negative. (Confidence = 42.98) 


In [35]:
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(model, dataset, orig_list[visual_idx], adv_list[visual_idx])

Original Prediction = Positive. (Confidence = 83.43) 


---------  After attack -------------
New Prediction = Positive. (Confidence = 40.76) 


In [46]:
## Save success
with open('attack_results_final.pkl', 'wb') as f:
    pickle.dump((test_list, orig_list, orig_label_list, adv_list, normalized_dist_list), f)
    


In [ ]:
## Cut cells:
goog_lm = LM()

src_word = dataset.dict['play']
nearest, nearest_dist = glove_utils.pick_most_similar_words(src_word, dist_mat,20)
nearest_w = [dataset.inv_dict[x] for x in nearest]
print('Closest to `%s` are %s' %(dataset.inv_dict[src_word], nearest_w))

prefix = 'is'
suffix = 'with'
lm_preds = goog_lm.get_words_probs(prefix, nearest_w, suffix)
print('most probable is ', nearest_w[np.argmax(lm_preds)])
